In [ ]:
## Import the different libraries
import os
import gc
import pandas as pd
import numpy as np

## Import the GBM libraries
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
import xgboost as xgb
from xgboost import XGBRegressor, DMatrix
import catboost as cb
from catboost import CatBoostRegressor, Pool
import lightgbm as lgb

## Import the validation metrics
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

from paramsearch import paramsearch
from itertools import product,chain

import pickle

## Import the plotting libraries
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.size']=7
plt.rcParams['savefig.dpi']=750

import time

import numbers

import warnings
warnings.filterwarnings(action='once')
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

## Import the shapley library
import shap

In [ ]:
gc.collect()

## Initialise path 
tempTemporaryPath= "./saveVariables/"
if not os.path.exists(tempTemporaryPath):
    os.makedirs(tempTemporaryPath)
    
methodCase=['xgb', 'cat', 'lgb']
for item in methodCase:
    if not os.path.exists(tempTemporaryPath + item + '_save'):
        os.makedirs(tempTemporaryPath + item + '_save')
    
    
    
## load data from parquet
# df_enc= pd.read_parquet('../data/preprocessEncData.parquet')
## load data from parquet
df_cat= pd.read_parquet('../data/preprocessWCatData.parquet')

for item in ['PruneDate', 'ThinDate', 'soil_final']:
    df_cat.loc[df_cat[item]<0, item]= np.nan

In [ ]:
## Choose depend variable
predictor= "SiteIndex"
## Remove some controversial features
list2Drop = ["Index","Age","Latitude", "Longitude", "SiteIndex" if predictor=="I300" else "I300"]
# df_enc= df_enc.drop(list2Drop, axis=1)
df_cat= df_cat.drop(list2Drop, axis=1)
## Inform about the names of the categorical features
catFeatures=['ThinType', 'Clone', 'ThinClass', 'PruneClass', 'Seedlot.Planting.Stock', 
             'Seedlot.Planting.Stock.Type', 'SeedlotCod', 'soil_final']

In [ ]:
## Remove the NaN from categorical features
df_cat.update(df_cat[catFeatures].fillna(-1000))
df_cat[catFeatures]=df_cat[catFeatures].astype(int)
for item in catFeatures:
    df_cat[item]=df_cat[item].astype("category").cat.codes + 1
    
## Do a copy, for later estimations, and remove the missing dependant variable observations
df_cat_copy= df_cat
df_cat = df_cat[pd.notnull(df_cat[predictor])]

In [ ]:
# split data into X and y
X_cat = df_cat.iloc[:,1:]
Y = df_cat[[predictor]]

cat_features_index= [X_cat.columns.get_loc(c) for c in catFeatures if c in X_cat]

# split categorical data into train and test sets
seedTest = 6969
test_size = 0.30
Xcat_train, Xcat_test, Ycat_train, Ycat_test = train_test_split(X_cat, Y, test_size=test_size, random_state=seedTest)

In [ ]:
for item in catFeatures:
    print(item)
    print(len(df_cat[item].unique()))

In [ ]:
with open("saveModels/datasetCategoricalSI.dat", "wb") as f:
    pickle.dump(df_cat, f)

###################################################################################
###################################################################################
#############################     END DATA PREPARATION     ###############################
###################################################################################
###################################################################################

###################################################################################
###################################################################################
##########################     GRIDSEARCH PARAMETERS TUNING     #########################
###################################################################################
###################################################################################

# XGBoost Tuning

In [ ]:
## Matrices for XGBoost
all_matrice = xgb.DMatrix(data=X_cat, label=Y)
train_matrice = xgb.DMatrix(Xcat_train, label = Ycat_train)
test_matrice = xgb.DMatrix(Xcat_test, label = Ycat_test)

In [ ]:
# param_XGB = {"max_depth": [3, 4, 5], 
#              "min_child_weight" : [4, 5, 6],
#              "num_round": [2000, 5000, 10000],
#              "learning_rate": [0.01, 0.2, 0.9],
#              "gamma": [0.0, 0.2, 0.4],
#              'tree_method': 'gpu_hist'
#             }

param_XGB = {"max_depth": 4, 
             "min_child_weight" : 3,
             "num_round": 7000,
             "learning_rate": [0.05, 0.1],
             "gamma": 1.0,
             'tree_method': 'gpu_hist'
            }

In [ ]:
def xgboost_param_tune(params, all_matrix, n_splits=3):
    ps = paramsearch(params)
    for prms in chain(#ps.grid_search(['max_depth', 'min_child_weight']),
                      #ps.grid_search(["gamma"])
                      #ps.grid_search(['num_round'])
                      ps.grid_search(['learning_rate'])
                      ):
        
            print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
            print("%%%%%%%% NEW TRAINING ROUND %%%%%%%%%%")
            print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")      
        
#         paramDone=[items for items in ps.results]
#         if prms in paramDone[1]:
#             print("params: " + str(prms) + " results: " + str(paramDone[0]))
#         else:
#             print("params: " + str(prms))

            startTime= time.time()
            print("params: " + str(prms))
            cv_data = xgb.cv(
                params= prms,
                dtrain=all_matrix,
                nfold= n_splits,
                num_boost_round=prms["num_round"],
                seed= 42,
                early_stopping_rounds=100
            )
            with open('./saveVariables/paramsXGB.pickle', 'wb') as handle:
                pickle.dump(prms, handle, protocol=pickle.HIGHEST_PROTOCOL)
            with open('./saveVariables/cvXGB.pickle', 'wb') as handle:
                pickle.dump(cv_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

            min_indx=cv_data.index[cv_data['test-rmse-mean']-cv_data['train-rmse-mean']>0.1]
            
            if len(min_indx)==0:
                res= cv_data['test-rmse-mean'].min()
                trainRMSE= cv_data.loc[cv_data['test-rmse-mean'].idxmin(), 'train-rmse-mean']
            else:
                res=cv_data.loc[min_indx[0], 'test-rmse-mean']
                trainRMSE= cv_data.loc[min_indx[0], 'train-rmse-mean']
                prms.update({"num_round": min_indx[0]})
                print(min_indx[0])

            print(time.time()- startTime)
            print("res: " + str(res))
            
            print("train: " + str(trainRMSE))
                  
            
            # save the crossvalidation result so that future iterations can reuse the best parameters
            ps.register_result(-res,prms)

            with open('./saveVariables/psXGB.pickle', 'wb') as handle:
                pickle.dump(ps, handle, protocol=pickle.HIGHEST_PROTOCOL)

    return(ps.bestparam())

In [ ]:
startTime=time.time()
bestparams = xgboost_param_tune(param_XGB, train_matrice)
print(time.time()- startTime)

In [ ]:
bestparams_XGB=bestparams
bestparams_XGB

### Fit Training XGB model

In [ ]:
startTime=time.time()
modelXGB = xgb.train(bestparams_XGB, train_matrice, num_boost_round=bestparams_XGB["num_round"])
print(time.time()- startTime)

In [ ]:
## Save model
modelXGB.dump_model('xgboost_raw.txt')
with open("saveModels/xgbModel_training.dat", "wb") as f:
    pickle.dump(modelXGB, f)   

In [ ]:
## Load model
with open("saveModels/xgbModel_training.dat", "rb") as f:
    modelXGB= pickle.load(f)

In [ ]:
## Evaluate the model
startTime=time.time()
predictionXGB= modelXGB.predict(test_matrice)
print(time.time()- startTime)
print("R2: " + str(r2_score(predictionXGB, Ycat_test)))
print("RMSE: " + str(np.sqrt(mean_squared_error(predictionXGB, Ycat_test))))

predictionXGB2= modelXGB.predict(train_matrice)
print("R2: " + str(r2_score(predictionXGB2, Ycat_train)))
print("RMSE: " + str(np.sqrt(mean_squared_error(predictionXGB2, Ycat_train))))

predictionXGB3= modelXGB.predict(all_matrice)
print("R2: " + str(r2_score(predictionXGB3, Y)))
print("RMSE: " + str(np.sqrt(mean_squared_error(predictionXGB3, Y))))

### Train final XGB model

In [ ]:
## We remove the gpu for the final training, the Sharpley method crashes on gpu_hist
bestparams_XGB['tree_method']= 'hist'

startTime=time.time()
modelXGB_final = xgb.train(bestparams_XGB, all_matrice, num_boost_round=bestparams_XGB["num_round"])
print(time.time()- startTime)


In [ ]:
with open("saveModels/xgboostModel.dat", "wb") as f:
    pickle.dump(modelXGB_final, f)

###########################################################
###########################################################
###########################################################

# Catboost Tuning

In [ ]:
## Pools for Catboost
all_pool= Pool(data=X_cat, label=Y, cat_features= cat_features_index, has_header=True)
train_pool = Pool(data=Xcat_train, label=Ycat_train, cat_features= cat_features_index, has_header=True)
test_pool = Pool(data=Xcat_test, label=Ycat_test, cat_features= cat_features_index, has_header=True)

In [ ]:
gc.collect()

In [ ]:
params_CAT = {'depth': 5,
              'iterations': 12000,
              'learning_rate':0.1,
              'l2_leaf_reg': 5,
              'one_hot_max_size': 10,
              'task_type':'GPU'
             }

In [ ]:
def catboost_param_tune(params, all_pool, n_splits=3):
    ps = paramsearch(params)
    for prms in chain(#ps.grid_search(['one_hot_max_size'])
                      #ps.grid_search(['depth'])
                      ps.grid_search(['l2_leaf_reg'])
                      #ps.grid_search(['learning_rate'])
                      ):
        
#         paramDone=[items for items in ps.results]
#         if prms in paramDone[1]:
#             print("params: " + str(prms) + " results: " + str(paramDone[0]))
#         else:
#             print("params: " + str(prms))

            print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
            print("%%%%%%%% NEW TRAINING ROUND %%%%%%%%%%")
            print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%") 

            print("params: " + str(prms))
    
            startTime=time.time()
            cv_data = cb.cv(
                all_pool,
                prms,
                fold_count= n_splits,
                partition_random_seed= 42,
                early_stopping_rounds=100,
                plot=False,
                verbose=False
            )
            with open('./saveVariables/paramsCAT.pickle', 'wb') as handle:
                pickle.dump(prms, handle, protocol=pickle.HIGHEST_PROTOCOL)
            with open('./saveVariables/cvCAT.pickle', 'wb') as handle:
                pickle.dump(cv_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
           

            
            min_indx=cv_data.index[cv_data['test-RMSE-mean']-cv_data['train-RMSE-mean']>0.1]
            
            if len(min_indx)==0:
                res= cv_data['test-RMSE-mean'].min()
                trainRMSE= cv_data.loc[cv_data['test-RMSE-mean'].idxmin(), 'train-RMSE-mean']
            else:
                res=cv_data.loc[min_indx[0], 'test-RMSE-mean']
                trainRMSE= cv_data.loc[min_indx[0], 'train-RMSE-mean']
                prms.update({"iterations": min_indx[0]})
                print(min_indx[0])
            
            print(time.time()- startTime)
            print("res: " + str(res))
            print("train: " + str(trainRMSE))
                
            # save the crossvalidation result so that future iterations can reuse the best parameters
            ps.register_result(-res,prms)

            with open('./saveVariables/psCAT.pickle', 'wb') as handle:
                pickle.dump(ps, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return ps.bestparam()


In [ ]:
startTime=time.time()
bestparams = catboost_param_tune(params_CAT, train_pool)
print(time.time()- startTime)

In [ ]:
# bestparams_CAT = {'depth': 5,
#               'iterations': 19829,
#               'learning_rate':0.1,
#               'l2_leaf_reg': 50,
#               'one_hot_max_size': 10,
#               'task_type':'GPU'
#              }

bestparams_CAT = bestparams
bestparams_CAT

### Fit Training CAT model

In [ ]:
## Train the catboost model
startTime=time.time()
modelCAT = cb.train(train_pool, bestparams_CAT, verbose=False)
print(time.time()-startTime)


In [ ]:
## Save model
with open("saveModels/catModel_training.dat", "wb") as f:
    pickle.dump(modelCAT, f)     

In [ ]:
## Load model
with open("saveModels/catModel_training.dat", "rb") as f:
    modelCAT= pickle.load(f)

In [ ]:
## Validate the model

startTime=time.time()
predictionCAT= modelCAT.predict(test_pool)
print(time.time()-startTime)
print("R2_test: " + str(r2_score(predictionCAT, Ycat_test)))
print("RMSE_test: " + str(np.sqrt(mean_squared_error(predictionCAT, Ycat_test))))

print("R2_train: " + str(r2_score(modelCAT.predict(train_pool), Ycat_train)))
print("RMSE_train: " + str(np.sqrt(mean_squared_error(modelCAT.predict(train_pool), Ycat_train))))

### Train final CAT model

In [ ]:
## Train the final model for prediction
startTime=time.time()
modelCAT_final = cb.train(all_pool, bestparams_CAT, verbose=False)
print(time.time()-startTime)

In [ ]:
## Save model
with open("saveModels/catboostModel.dat", "wb") as f:
    pickle.dump(modelCAT_final, f)
    
## Load model
# with open("saveModels/catboostModel.dat", "rb") as f:
#     modelCAT_final= pickle.load(f)

###########################################################
###########################################################
###########################################################

# LightGBM Tuning

In [ ]:
catFeatures=['ThinType', 'Clone', 'ThinClass', 'PruneClass', 'Seedlot.Planting.Stock', 
             'Seedlot.Planting.Stock.Type', 'soil_final']
cat_features_index= [X_cat.columns.get_loc(c) for c in catFeatures if c in X_cat]

In [ ]:
gc.collect()

In [ ]:
## Dataset for LightGBM
all_dataset= lgb.Dataset(data=X_cat, label=Y, categorical_feature= cat_features_index, free_raw_data=False)

# train_dataset = lgb.Dataset(data=Xcat_train, label=Ycat_train, categorical_feature= cat_features_index, free_raw_data=False)

train_dataset = lgb.Dataset(data=Xcat_train, label=Ycat_train)
test_dataset = lgb.Dataset(data=Xcat_test, label=Ycat_test, categorical_feature= cat_features_index, free_raw_data=False)

# train_partial_dataset = lgb.Dataset(data=Xcat_train_partial, label=Ycat_train_partial, categorical_feature= cat_features_index, free_raw_data=False)
# valid_dataset = lgb.Dataset(data=Xcat_valid, label=Ycat_valid, categorical_feature= cat_features_index, free_raw_data=False)

In [ ]:
# params_LGB = {'max_depth':[4, 5, 6],
#               'n_estimators':15000,
#               'learning_rate':[0.05, 0.1, 0.15],
#               'num_leaves':[10, 12, 14],
#               'objective': 'regression',
#               'metric': 'rmse',
#               'device':'gpu',
#               'gpu_platform_id' :0,
#               'gpu_device_id':0 
#              }

params_LGB = {'max_depth':4,
              'n_estimators':25000,
              'learning_rate':0.1,
              'num_leaves':10,
              'min_data_in_leaf': [100, 50, 40, 30, 20, 15, 10, 5, 2],
              'objective': 'regression',
              'metric': 'rmse',
              'is_training_metric': True,
              'device':'gpu',
              'gpu_platform_id' :0,
              'gpu_device_id':0 
             }

In [ ]:
def lightgbm_param_tune(params, X, Y, cat_features_index, n_splits=3):
    ps = paramsearch(params)
    for prms in chain(ps.grid_search(['min_data_in_leaf'])
                      #ps.grid_search(['num_leaves'])
                      #ps.grid_search(['learning_rate'])
                      ):
#         paramDone=[items for items in ps.results]
#         if prms in paramDone[1]:
#             print("params: " + str(prms) + " results: " + str(paramDone[0]))
#         else:
#             print("params: " + str(prms))
            print("params: " + str(prms))
    
            startTime=time.time()
            
            
            setValidName= ['test-RMSE', 'train-RMSE']

            cv_data= pd.DataFrame()
            kf= KFold(n_splits, random_state=42, shuffle= True)
            for train_index, test_index in kf.split(X):
                Xlocal_train, Xlocal_valid= X.iloc[train_index], X.iloc[test_index]
                Ylocal_train, Ylocal_valid= Y.iloc[train_index], Y.iloc[test_index]

                train_dataset = lgb.Dataset(data=Xlocal_train, label=Ylocal_train)
                valid_dataset = lgb.Dataset(data=Xlocal_valid, label=Ylocal_valid)
                setValid= [valid_dataset, train_dataset]
                evalRes={}

                cvLGB= lgb.train(prms, train_dataset, valid_sets= setValid, valid_names= setValidName,
                                 early_stopping_rounds=100, verbose_eval=False, evals_result=evalRes,
                                categorical_feature= cat_features_index)

                dfRes= pd.DataFrame.from_dict(evalRes)
                dfRes= dfRes.transpose()
                dfRes= dfRes.rmse.apply(pd.Series)
                dfRes= dfRes.transpose()
                cv_data= pd.concat([cv_data, dfRes], axis=1)

            cv_data=cv_data.groupby(by=cv_data.columns, axis=1).apply(lambda g: g.mean(axis=1) if isinstance(g.iloc[0,0], numbers.Number) else g.iloc[:,0])

            min_indx=cv_data.index[cv_data['test-RMSE']-cv_data['train-RMSE']>0.1]



            with open('./saveVariables/paramsLGB.pickle', 'wb') as handle:
                pickle.dump(prms, handle, protocol=pickle.HIGHEST_PROTOCOL)
            with open('./saveVariables/cvLGB.pickle', 'wb') as handle:
                pickle.dump(cv_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
               
            print('Actual')
            print(cv_data['test-RMSE'].min())
            print(cv_data.loc[cv_data['test-RMSE'].idxmin(), 'train-RMSE'])
            
            if len(min_indx)==0:
                res= cv_data['test-RMSE'].min()
                trainRMSE= cv_data.loc[cv_data['test-RMSE'].idxmin(), 'train-RMSE']
            else:
                res=cv_data.loc[min_indx[0], 'test-RMSE']
                trainRMSE= cv_data.loc[min_indx[0], 'train-RMSE']
                prms.update({"n_estimators": min_indx[0]})
                print(min_indx[0])

                
            print(time.time()- startTime)
            print("res: " + str(res))
            
            print("train: " + str(trainRMSE))
            print('---------Next---------')
            # save the crossvalidation result so that future iterations can reuse the best parameters
            ps.register_result(-res,prms)

            with open('./saveVariables/psLGB.pickle', 'wb') as handle:
                pickle.dump(ps, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return ps.bestparam()


In [ ]:
startTime=time.time()
best_params= lightgbm_param_tune(params_LGB, Xcat_train, Ycat_train, cat_features_index=cat_features_index)
print(time.time()-startTime)


In [ ]:
bestparams_LGB=best_params
bestparams_LGB

### Fit Training LGB model

In [ ]:
def custom_evalR2(y_true, y_pred):
    return("r2", [np.sqrt(mean_squared_error(y_true, y_pred.get_label())), r2_score(y_true, y_pred.get_label())], True)

In [ ]:
## Measure R2 during training
startTime=time.time()
evalFinalRes={}
modelLGB = lgb.train(
    params=bestparams_LGB, 
    train_set= train_dataset,
    valid_sets= train_dataset,
    feval= custom_evalR2,
    verbose_eval= False, 
    evals_result=evalFinalRes,
    categorical_feature= cat_features_index
)
print(time.time()-startTime)

In [ ]:
rmseRes= pd.DataFrame.from_dict(evalFinalRes)
metricRes= rmseRes.transpose()
rmseRes= metricRes.rmse.apply(pd.Series)
r2Res= metricRes.r2.apply(pd.Series)
rmseRes= rmseRes.transpose()
r2Res=r2Res.transpose()
print("RMSE_train: " + str(rmseRes.iloc[len(rmseRes)-1]) + "\n R2_train: " + str(r2Res.iloc[len(r2Res)-1])) 


In [ ]:
modelLGB.save_model("saveModels/lightgbmModel.cbm")

In [ ]:
## Evaluate the moel
startTime=time.time()
# Get predictions
predsLGB = modelLGB.predict(Xcat_test)
print(time.time()-startTime)

print("R2: " + str(r2_score(Ycat_test, predsLGB)))
print("RMSE: " + str(np.sqrt(mean_squared_error(predsLGB, Ycat_test))))



In [ ]:
predsLGB2 = modelLGB.predict(X_cat)
predsLGB3 = modelLGB.predict(Xcat_train)

print("R2: " + str(r2_score(Y, predsLGB2)))
print("RMSE: " + str(np.sqrt(mean_squared_error(predsLGB2, Y))))

print("R2: " + str(r2_score(predsLGB3, Ycat_train)))
print("RMSE: " + str(np.sqrt(mean_squared_error(predsLGB3, Ycat_train))))


### Train final LGB model

In [ ]:
## Train final model for prediction
startTime=time.time()
modelLGB_final = lgb.train(
    params=bestparams_LGB, 
    train_set= all_dataset,
    verbose_eval= False, 
    categorical_feature= cat_features_index
)
print(time.time()-startTime)


In [ ]:
## Save model
with open("saveModels/lightgbmModel.dat", "wb") as f:
    pickle.dump(modelLGB_final, f)

## Load model
# with open("saveModels/lightgbmModel.dat", "rb") as f:
#     modelLGB_final= pickle.load(f)

###################################################################################
###################################################################################
##########################     END PARAMETERS TUNING     ##########################
###################################################################################
###################################################################################